## Import data

In [1]:
import os
import torch
import relbench

relbench.__version__

'1.1.0'

In [2]:
import numpy as np

from torch.nn import BCEWithLogitsLoss, L1Loss
from relbench.datasets import get_dataset
from relbench.tasks import get_task

dataset = get_dataset("rel-f1", download=True)
task = get_task("rel-f1", "driver-position", download=True)

train_table = task.get_table("train")
val_table = task.get_table("val")
test_table = task.get_table("test")

out_channels = 1
loss_fn = L1Loss()
tune_metric = "mae"
higher_is_better = False

Let's check out the training table just to make sure it looks fine.

In [3]:
train_table

Table(df=
           date  driverId  position
0    2004-07-05        10     10.75
1    2004-07-05        47     12.00
2    2004-03-07         7     15.00
3    2004-01-07        10      9.00
4    2003-09-09        52     13.00
...         ...       ...       ...
7448 1995-08-22        96     15.75
7449 1975-06-08       228      8.00
7450 1965-05-31       418     16.00
7451 1961-08-20       467     37.00
7452 1954-05-29       677     30.00

[7453 rows x 3 columns],
  fkey_col_to_pkey_table={'driverId': 'drivers'},
  pkey_col=None,
  time_col=date)

Note that to load the data we did not require any deep learning libraries. Now we introduce the PyTorch Frame library, which is useful for encoding individual tables into initial node features.

In [4]:
import os
import math
import numpy as np
from tqdm import tqdm

import torch
import torch_geometric
import torch_frame

# Some book keeping
from torch_geometric.seed import seed_everything

# seed_everything(42)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)  # check that it's cuda if you want it to run in reasonable time!
root_dir = "./data"

cuda


The first big move is to build a graph out of the database. Here we use our pre-prepared conversion function.

The source code can be found at: https://github.com/snap-stanford/relbench/blob/main/relbench/modeling/graph.py

Each node in the graph corresonds to a single row in the database. Crucially, PyTorch Frame stores whole tables as objects in a way that is compatibile with PyG minibatch sampling, meaning we can sample subgraphs as in https://arxiv.org/abs/1706.02216, and retrieve the relevant raw features.

PyTorch Frame also stores the `stype` (i.e., modality) of each column, and any specialized feature encoders (e.g., text encoders) to be used later. So we need to configure the `stype` for each column, for which we use a function that tries to automatically detect the `stype`.

In [5]:
from relbench.modeling.utils import get_stype_proposal

db = dataset.get_db()
col_to_stype_dict = get_stype_proposal(db)
col_to_stype_dict



Loading Database object from /home/lema/.cache/relbench/rel-f1/db...
Done in 0.10 seconds.


{'constructors': {'constructorId': <stype.numerical: 'numerical'>,
  'constructorRef': <stype.text_embedded: 'text_embedded'>,
  'name': <stype.text_embedded: 'text_embedded'>,
  'nationality': <stype.text_embedded: 'text_embedded'>},
 'circuits': {'circuitId': <stype.numerical: 'numerical'>,
  'circuitRef': <stype.text_embedded: 'text_embedded'>,
  'name': <stype.text_embedded: 'text_embedded'>,
  'location': <stype.text_embedded: 'text_embedded'>,
  'country': <stype.text_embedded: 'text_embedded'>,
  'lat': <stype.numerical: 'numerical'>,
  'lng': <stype.numerical: 'numerical'>,
  'alt': <stype.numerical: 'numerical'>},
 'races': {'raceId': <stype.numerical: 'numerical'>,
  'year': <stype.categorical: 'categorical'>,
  'round': <stype.numerical: 'numerical'>,
  'circuitId': <stype.numerical: 'numerical'>,
  'name': <stype.text_embedded: 'text_embedded'>,
  'date': <stype.timestamp: 'timestamp'>,
  'time': <stype.timestamp: 'timestamp'>},
 'drivers': {'driverId': <stype.numerical: 'n

In [6]:
db.table_dict.keys()

dict_keys(['constructors', 'circuits', 'races', 'drivers', 'constructor_standings', 'standings', 'constructor_results', 'results', 'qualifying'])

In [7]:
table = db.table_dict["circuits"].df
# table[table["alt"].isnull()]
table
# set missing alt
table.loc[75, "alt"] = 15
table.loc[76, "alt"] = 2
table.loc[22, "alt"] = 634

table

,circuitId,circuitRef,name,location,country,lat,lng,alt
0,0,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10.0
1,1,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18.0
2,2,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7.0
3,3,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109.0
4,4,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130.0
...,...,...,...,...,...,...,...,...
72,72,portimao,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,108.0
73,73,mugello,Autodromo Internazionale del Mugello,Mugello,Italy,43.99750,11.37190,255.0
74,74,jeddah,Jeddah Corniche Circuit,Jeddah,Saudi Arabia,21.63190,39.10440,15.0
75,75,losail,Losail International Circuit,Al Daayen,Qatar,25.49000,51.45420,15.0


In [8]:
for table_name in db.table_dict.keys():
    print(table_name)
    print(db.table_dict[table_name].df)
    df_ = db.table_dict[table_name].df
    nan_columns = df_.columns[df_.isna().any()].tolist()
    print(nan_columns)

constructors
     constructorId constructorRef            name nationality
0                0        mclaren         McLaren     British
1                1     bmw_sauber      BMW Sauber      German
2                2       williams        Williams     British
3                3        renault         Renault      French
4                4     toro_rosso      Toro Rosso     Italian
..             ...            ...             ...         ...
206            206          manor  Manor Marussia     British
207            207           haas    Haas F1 Team    American
208            208   racing_point    Racing Point     British
209            209     alphatauri      AlphaTauri     Italian
210            210         alpine  Alpine F1 Team      French

[211 rows x 4 columns]
[]
circuits
    circuitId   circuitRef                                  name  \
0           0  albert_park        Albert Park Grand Prix Circuit   
1           1       sepang          Sepang International Circuit   
2  

In [9]:
table = db.table_dict["results"].df
table[~table["fastestLap"].isnull()]

,resultId,raceId,driverId,constructorId,number,grid,position,positionOrder,points,laps,milliseconds,fastestLap,rank,statusId,date
18109,18109,713,43,6,17.0,18,13.0,13,0.0,56,NaN,35.0,14.0,12,2004-03-07 00:00:00
18110,18110,713,44,16,19.0,16,14.0,14,0.0,55,NaN,15.0,18.0,13,2004-03-07 00:00:00
18111,18111,713,12,14,12.0,11,NaN,15,0.0,44,NaN,11.0,12.0,5,2004-03-07 00:00:00
18112,18112,713,42,6,16.0,13,12.0,12,0.0,56,NaN,41.0,15.0,12,2004-03-07 00:00:00
18113,18113,713,45,17,20.0,20,NaN,17,0.0,43,NaN,10.0,19.0,62,2004-03-07 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20318,20318,819,1,1,6.0,8,5.0,5,4.0,55,5669667.0,54.0,7.0,1,2009-11-01 11:00:00
20319,20319,819,21,22,23.0,4,4.0,4,5.0,55,5666149.0,54.0,4.0,1,2009-11-01 11:00:00
20320,20320,819,17,22,22.0,5,3.0,3,6.0,55,5661881.0,49.0,6.0,1,2009-11-01 11:00:00
20321,20321,819,16,8,14.0,3,2.0,2,8.0,55,5661271.0,14.0,5.0,1,2009-11-01 11:00:00


If trying a new dataset, you should definitely check through this dict of `stype`s to check that look right, and manually change any mistakes by the auto-detection function.

Next we also define our text encoding model, which we use GloVe embeddings for speed and convenience. Feel free to try alternatives here.

In [10]:
# !pip install -U sentence-transformers # we need another package for text encoding
from typing import List, Optional
from sentence_transformers import SentenceTransformer
from torch import Tensor


class GloveTextEmbedding:
    def __init__(self, device: Optional[torch.device
                                       ] = None):
        self.model = SentenceTransformer(
            "sentence-transformers/average_word_embeddings_glove.6B.300d",
            device=device,
        )

    def __call__(self, sentences: List[str]) -> Tensor:
        return torch.from_numpy(self.model.encode(sentences))



In [ ]:
from torch_frame.config.text_embedder import TextEmbedderConfig
from relbench.modeling.graph import make_pkey_fkey_graph

text_embedder_cfg = TextEmbedderConfig(
    text_embedder=GloveTextEmbedding(device=device), batch_size=256
)

data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,  # speficied column types
    text_embedder_cfg=text_embedder_cfg,  # our chosen text encoder
    cache_dir=os.path.join(
        root_dir, f"rel-f1_materialized_cache"
    ),  # store materialized graph for convenience
)

We can now check out `data`, our main graph object. `data` is a heterogeneous and temporal graph, with node types given by the table it originates from.

We can also check out the TensorFrame for one table like this:

In [ ]:
data

HeteroData(
  constructors={ tf=TensorFrame([211, 3]) },
  circuits={ tf=TensorFrame([77, 7]) },
  races={
    tf=TensorFrame([820, 5]),
    time=[820],
  },
  drivers={ tf=TensorFrame([857, 6]) },
  constructor_standings={
    tf=TensorFrame([10170, 4]),
    time=[10170],
  },
  standings={
    tf=TensorFrame([28115, 4]),
    time=[28115],
  },
  constructor_results={
    tf=TensorFrame([9408, 2]),
    time=[9408],
  },
  results={
    tf=TensorFrame([20323, 11]),
    time=[20323],
  },
  qualifying={
    tf=TensorFrame([4082, 3]),
    time=[4082],
  },
  (races, f2p_circuitId, circuits)={ edge_index=[2, 820] },
  (circuits, rev_f2p_circuitId, races)={ edge_index=[2, 820] },
  (constructor_standings, f2p_raceId, races)={ edge_index=[2, 10170] },
  (races, rev_f2p_raceId, constructor_standings)={ edge_index=[2, 10170] },
  (constructor_standings, f2p_constructorId, constructors)={ edge_index=[2, 10170] },
  (constructors, rev_f2p_constructorId, constructor_standings)={ edge_index=[2, 1

In [ ]:
data["races"].tf

TensorFrame(
  num_cols=5,
  num_rows=820,
  categorical (1): ['year'],
  numerical (1): ['round'],
  timestamp (2): ['date', 'time'],
  embedding (1): ['name'],
  has_target=False,
  device='cpu',
)

This may be a little confusing at first, as in graph ML it is more standard to associate to the graph object `data` a tensor, e.g., `data.x` for which `data.x[idx]` is a 1D array/tensor storing all the features for node with index `idx`.

But actually this `data` object behaves similarly. For a given node type, e.g., `races` again, `data['races']` stores two pieces of information


In [ ]:
list(data["races"].keys())

['tf', 'time']

A `TensorFrame` object, and a timestamp for each node. The `TensorFrame` object acts analogously to the usual tensor of node features, and you can simply use indexing to retrieve the features of a single row (node), or group of nodes.

In [ ]:
data["races"].tf[10]

TensorFrame(
  num_cols=5,
  num_rows=1,
  categorical (1): ['year'],
  numerical (1): ['round'],
  timestamp (2): ['date', 'time'],
  embedding (1): ['name'],
  has_target=False,
  device='cpu',
)

In [ ]:
data["races"].tf[10:20]

TensorFrame(
  num_cols=5,
  num_rows=10,
  categorical (1): ['year'],
  numerical (1): ['round'],
  timestamp (2): ['date', 'time'],
  embedding (1): ['name'],
  has_target=False,
  device='cpu',
)

We can also check the edge indices between two different node types, such as `races` amd `circuits`. Note that the edges are also heterogenous, so we also need to specify which edge type we want to look at. Here we look at `f2p_curcuitId`, which are the directed edges pointing _from_ a race (the `f` stands for `foreign key`), _to_ the circuit at which te race happened (the `p` stands for `primary key`).

In [ ]:
data[("races", "f2p_circuitId", "circuits")]

{'edge_index': tensor([[  0,   1,   2,  ..., 817, 818, 819],
        [  8,   5,  18,  ...,  21,  17,  23]])}

Now we are ready to instantiate our data loaders. For this we will need to import PyTorch Geometric, our GNN library. Whilst we're at it let's add a seed.


## Settings and model

In [ ]:
default_settings = {
    "depth": 2,
    "num_sampled_neighbors": 128,
    "batch_size": 512,
    "temporal_strategy": "uniform",
    "num_layers": 2,
    "channels": 128,    
    "aggr":"sum",
    "id_awareness": False,
    "num_blocks": 1,
    "lr": 0.005,
    "epochs": 10
}

In [ ]:
# import pandas as pd	

# # reset the results file
# cols = ["id", "model", "val/test", "r2", "mae", "rmse", "date", "time", "where", "time_needed", "id_awareness", "epochs", "num_blocks", "num_sampled_neighbors", "depth", "batch_size", "temporal_strategy", "num_layers", "channels", "aggr"]
# df = pd.DataFrame(columns=cols)

# df.to_csv("results.csv", index=False)

In [ ]:
import pandas as pd	
def new_record(settings, val_test, errors, time_needed, where ="Leila laptop", which_model="Given model", file = "results", extension=""):
    file = file + "_" + extension + ".csv"
    try:
        df = pd.read_csv(file)
    except Exception as e:
        # # reset the results file
        cols = ["id", "model", "val/test", "r2", "mae", "rmse", "date", "time", "where", "time_needed", "id_awareness", "epochs", "num_blocks", "num_sampled_neighbors", "depth", "batch_size", "temporal_strategy", "num_layers", "channels", "aggr"]
        df = pd.DataFrame(columns=cols)

        # df.to_csv(file, index=False)
    d = {}
    for setting in settings:
        d[setting] = settings[setting]
    d["id"] = None
    d["model"] = which_model
    d["val/test"] = val_test
    d["where"] = where
    d["time_needed"] = time_needed
    
    current_date = pd.Timestamp.now().strftime("%d.%m.%Y")
    current_time = pd.Timestamp.now().strftime("%H:%M:%S")
    d["date"] = current_date
    d["time"] = current_time
    

    d["r2"] = float(errors["r2"])
    d["mae"] = float(errors["mae"])
    d["rmse"] = float(errors["rmse"])
    
    print(str(d))
    

    df2 = pd.DataFrame([d])

    df3 = pd.concat([df, df2], ignore_index=True,axis=0)
    
    df3.to_csv(file, index=False)

In [ ]:
# new_record(settings, "val", (0.0, 0.0, 0.0), 0.0)

In [ ]:
from relbench.modeling.graph import get_node_train_table_input, make_pkey_fkey_graph
from torch_geometric.loader import NeighborLoader

def get_loader(settings, data, task, train_table, val_table, test_table):
    loader_dict = {}

    for split, table in [
        ("train", train_table),
        ("val", val_table),
        ("test", test_table),
    ]:
        table_input = get_node_train_table_input(
            table=table,
            task=task,
        )
        entity_table = table_input.nodes[0]
        
        depth = settings["depth"]
        num_sampled_neighbors = settings["num_sampled_neighbors"]
        
        loader_dict[split] = NeighborLoader(
            data,
            num_neighbors=[
                num_sampled_neighbors for i in range(depth)
                # 128 for i in range(2)
                # 64 for i in range(2)
            ],  # we sample subgraphs of depth 2, 128 neighbors per node.
            time_attr="time",
            input_nodes=table_input.nodes,
            input_time=table_input.time,
            transform=table_input.transform,
            
            batch_size=settings["batch_size"],
            
            temporal_strategy=settings["temporal_strategy"],
            # If set to :obj:`"uniform"`, will sample uniformly across neighbors
            #     that fulfill temporal constraints.
            #     If set to :obj:`"last"`, will sample the last `num_neighbors` that
            #     fulfill temporal constraints.
            
            shuffle=split == "train",
            num_workers=0,
            persistent_workers=False,
        )
    return loader_dict, entity_table

In [ ]:
from modules import Model
def get_model_opt_epochs(settings, data, col_stats_dict):
    model = Model(
        data=data,
        col_stats_dict=col_stats_dict,
        num_layers=settings["num_layers"],
        channels=settings["channels"], # dim embeddingov (vmes), neodvisno
        out_channels=1,
        aggr=settings["aggr"],
        norm="batch_norm",
        id_awareness=settings["id_awareness"],
        
        num_blocks=settings["num_blocks"],
    ).to(device)


    # if you try out different RelBench tasks you will need to change these
    optimizer = torch.optim.Adam(model.parameters(), lr=settings["lr"])
    epochs = settings["epochs"]
    
    return model, optimizer, epochs

## Train

In [ ]:
def get_train_test(model, optimizer, loader_dict, entity_table):
    def train() -> float:
        model.train()

        loss_accum = count_accum = 0
        for batch in tqdm(loader_dict["train"]):
            batch = batch.to(device)

            optimizer.zero_grad()
            pred = model(
                batch,
                task.entity_table,
            )
            pred = pred.view(-1) if pred.size(1) == 1 else pred

            loss = loss_fn(pred.float(), batch[entity_table].y.float())
            loss.backward()
            optimizer.step()

        loss_accum += loss.detach().item() * pred.size(0)
        count_accum += pred.size(0)

        return loss_accum / count_accum


    @torch.no_grad()
    def test(loader: NeighborLoader) -> np.ndarray:
        model.eval()

        pred_list = []
        for batch in loader:
            batch = batch.to(device)
            pred = model(
                batch,
                task.entity_table,
            )
            pred = pred.view(-1) if pred.size(1) == 1 else pred
            pred_list.append(pred.detach().cpu())
        return torch.cat(pred_list, dim=0).numpy()
    
    return train, test

In [ ]:
import copy
import time
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18, "font.family": "serif"})

start_time = time.time()
num_repeats = 1
setting_name = "num_layers"
for i in range(num_repeats):
    for val in [2,4,6]:
        settings = copy.deepcopy(default_settings)
        settings[setting_name] = val
        settings["epochs"] = 50
        

        loader_dict, entity_table = get_loader(settings, data, task, train_table, val_table, test_table)
        model, optimizer, epochs = get_model_opt_epochs(settings, data, col_stats_dict)
        train, test = get_train_test(model, optimizer, loader_dict, entity_table)


        state_dict = None
        best_val_metric = -math.inf if higher_is_better else math.inf
        tls = []
        val_maes = []
        test_maes = []
        for epoch in range(1, epochs + 1):
            train_loss = train()
            val_pred = test(loader_dict["val"])
            
            # print(val_pred)
            # print(val_table)
            # exit()
            
            val_metrics = task.evaluate(val_pred, val_table)
            print(f"Epoch: {epoch:02d}, Train loss: {train_loss}, Val metrics: {val_metrics}")
            
            tls.append(train_loss)
            val_maes.append(val_metrics["mae"])
            
            test_pred = test(loader_dict["test"])
            test_metrics = task.evaluate(test_pred)
            test_maes.append(test_metrics["mae"])
            

            if (higher_is_better and val_metrics[tune_metric] > best_val_metric) or (
                not higher_is_better and val_metrics[tune_metric] < best_val_metric
            ):
                best_val_metric = val_metrics[tune_metric]
                state_dict = copy.deepcopy(model.state_dict())

        time_needed = int(round(time.time() - start_time))
        
    
        
        # plt.title(f"Train and validation MAE for {val} layers")
        # plt.plot(tls, "--o", color = "blue", label = "Train MAE")        
        # plt.plot(val_maes, "--o", color = "orange", label = "Validation MAE")
        # plt.plot(test_maes, "--o", color = "green", label = "Test MAE")
        # plt.legend()
        # plt.savefig(f"curve_{setting_name}_{val}.png")
        # plt.clf()


        model.load_state_dict(state_dict)
        val_pred = test(loader_dict["val"])
        val_metrics = task.evaluate(val_pred, val_table)
        print(f"Best Val metrics: {val_metrics}")

        test_pred = test(loader_dict["test"])
        test_metrics = task.evaluate(test_pred)
        print(f"Best test metrics: {test_metrics}")
        
        new_record(settings, "val", val_metrics, time_needed, which_model="Given model", where="Leila laptop", file = "results", extension=setting_name)
        new_record(settings, "test", test_metrics, time_needed, which_model="Given model", where="Leila laptop", file = "results", extension=setting_name)


100%|██████████| 15/15 [00:07<00:00,  1.96it/s]
/home/lema/Documents/faks/MLG/machine_learning_on_graphs/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch: 01, Train loss: 6.627116680145264, Val metrics: {'r2': -0.20933774905728075, 'mae': np.float64(4.293905221356817), 'rmse': np.float64(5.098284823593611)}


/home/lema/Documents/faks/MLG/machine_learning_on_graphs/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
100%|██████████| 15/15 [00:06<00:00,  2.32it/s]
/home/lema/Documents/faks/MLG/machine_learning_on_graphs/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch: 02, Train loss: 5.440690994262695, Val metrics: {'r2': -0.37248727301718243, 'mae': np.float64(4.3288013964075205), 'rmse': np.float64(5.431308263048824)}


/home/lema/Documents/faks/MLG/machine_learning_on_graphs/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
100%|██████████| 15/15 [00:06<00:00,  2.38it/s]
/home/lema/Documents/faks/MLG/machine_learning_on_graphs/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


...

Epoch: 50, Train loss: 4.262711048126221, Val metrics: {'r2': 0.1765614083375473, 'mae': np.float64(3.3744353468289114), 'rmse': np.float64(4.206935931858428)}


/home/lema/Documents/faks/MLG/machine_learning_on_graphs/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/lema/Documents/faks/MLG/machine_learning_on_graphs/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Best Val metrics: {'r2': 0.22621073915833845, 'mae': np.float64(3.2529999704303627), 'rmse': np.float64(4.078135378797415)}
Best test metrics: {'r2': 0.04465358924827678, 'mae': np.float64(4.123846813042959), 'rmse': np.float64(5.092725802789871)}
{'depth': 2, 'num_sampled_neighbors': 128, 'batch_size': 512, 'temporal_strategy': 'uniform', 'num_layers': 6, 'channels': 128, 'aggr': 'sum', 'id_awareness': False, 'num_blocks': 1, 'lr': 0.005, 'epochs': 50, 'id': None, 'model': 'Given model', 'val/test': 'val', 'where': 'Leila laptop', 'time_needed': 1667, 'date': '13.12.2024', 'time': '21:33:28', 'r2': 0.22621073915833845, 'mae': 3.2529999704303627, 'rmse': 4.078135378797415}
{'depth': 2, 'num_sampled_neighbors': 128, 'batch_size': 512, 'temporal_strategy': 'uniform', 'num_layers': 6, 'channels': 128, 'aggr': 'sum', 'id_awareness': False, 'num_blocks': 1, 'lr': 0.005, 'epochs': 50, 'id': None, 'model': 'Given model', 'val/test': 'test', 'where': 'Leila laptop', 'time_needed': 1667, 'date'

/home/lema/Documents/faks/MLG/machine_learning_on_graphs/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


<Figure size 640x480 with 0 Axes>

In [ ]:
# new_record(settings, "val", val_metrics, time_needed)
# new_record(settings, "test", test_metrics, time_needed)

In [ ]:
# with open("metrics.out", "a") as f:
#     f.write(f"Best Val metrics: {list(map(float, val_metrics.values()))}\n".replace(",",""))
#     f.write(f"Best test metrics: {list(map(float, test_metrics.values()))}\n".replace(",",""))